ДЗ
===
Использовать датасет MovieLens
Использовать любые модели
Добиться целевого значения метрики RMSE на тестовой выборке 0.87 и ниже.

http://surpriselib.com

In [2]:
import pandas as pd
import numpy as np
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
from surprise import SVD
from surprise import SVDpp

In [ ]:
pip install surprise

Использовать датасет на 1 млн записей не получилось, при обучении умирает ядро

In [3]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.drop(columns=['timestamp'], inplace=True)
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [6]:
df = pd.merge(movies, ratings, on='movieId', how='inner')
df.tail()

,movieId,title,genres,userId,rating
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184,4.0
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184,3.5
100833,193585,Flint (2017),Drama,184,3.5
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184,3.5
100835,193609,Andrew Dice Clay: Dice Rules (1991),Comedy,331,4.0


In [20]:
['Ok' for x in df.isnull().any() if x == False ]

['Ok', 'Ok', 'Ok', 'Ok', 'Ok']

Создаем датасет для Surprise
========

In [7]:
dataset = pd.DataFrame({
    'uid': df.userId,
    'iid': df.title,
    'rating': df.rating
})
dataset.head()

,uid,iid,rating
0,1,Toy Story (1995),4.0
1,5,Toy Story (1995),4.0
2,7,Toy Story (1995),4.5
3,15,Toy Story (1995),2.5
4,17,Toy Story (1995),4.5


In [8]:
reader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))
data = Dataset.load_from_df(dataset, reader)

In [9]:
trainset, testset = train_test_split(data, test_size=0.15)

User-based
======

In [12]:
k = list(range(10, 100, 10))
best_rmse = 10

for i in tqdm(k):
    algo_user_true = KNNWithMeans(k=i , sim_options={'name': 'pearson_baseline', 'user_based': True})
    algo_user_true.fit(trainset)
    test_pred_user_true = algo_user_true.test(testset)
    acc = accuracy.rmse(test_pred_user_true, verbose=True)
    if acc< best_rmse:
        best_rmse = acc
        best_k = i
print('best params {} is rmse = {}'.format(best_k, best_rmse))       

/Users/andreysemeshin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8859
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8831
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8828
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8830
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8833
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8835
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8836
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done co

item-based
========

In [14]:
k = list(range(10, 100, 10))
best_rmse = 10

for i in tqdm(k):
    algo_user_true = KNNWithMeans(k=i , sim_options={'name': 'pearson_baseline', 'user_based': False})
    algo_user_true.fit(trainset)
    test_pred_user_true = algo_user_true.test(testset)
    acc = accuracy.rmse(test_pred_user_true, verbose=True)
    if acc< best_rmse:
        best_rmse = acc
        best_k = i
print('best params {} is rmse = {}'.format(best_k, best_rmse))     

/Users/andreysemeshin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8856
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8767
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8740
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8735
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8733
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8732
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8732
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done co

SVD
======

In [15]:
algo_svd = SVD()
algo_svd.fit(trainset)
test_pred_svd = algo_svd.test(testset)
accuracy.rmse(test_pred_svd, verbose=True)

RMSE: 0.8630


0.8630014635881799

SVD++
=======

In [18]:
algo_svdpp = SVDpp()
algo_svdpp.fit(trainset)
test_pred_svdpp = algo_svdpp.test(testset)
accuracy.rmse(test_pred_svdpp, verbose=True)

RMSE: 0.8492


0.8491673603405163

Итог
======

Метод SVD показал результат - 0.863001463

Лучший результат дал метод SVD++ - 0.84916736, однако время обучения модели составляет 15-20 минут